In [1]:
import pandas as pd
import numpy as np
import codecs

In [2]:
companies = pd.read_csv('companies.txt',encoding='ISO-8859-1',sep='\t')
companies.permalink = companies.permalink.str.encode('ISO-8859-1').str.decode('ascii','ignore')
companies.name = companies.name.str.encode('ISO-8859-1').str.decode('ascii','ignore')

In [3]:
companies.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010


In [4]:
rounds2 = pd.read_csv('rounds2.csv',encoding='ISO-8859-1')
rounds2.company_permalink = rounds2.company_permalink.str.encode('ISO-8859-1').str.decode('ascii','ignore')

In [5]:
rounds2.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [6]:
companies['permalink']=companies['permalink'].str.lower()
print(len(companies['permalink'].unique()))

66368


In [7]:
companies['permalink'].str.lower().describe()

count                   66368
unique                  66368
top       /organization/-fame
freq                        1
Name: permalink, dtype: object

In [8]:
rounds2['company_permalink']=rounds2['company_permalink'].str.lower()
print(len(rounds2['company_permalink'].unique()))

66368


In [9]:
rounds2['company_permalink'].str.lower().describe()

count                       114949
unique                       66368
top       /organization/solarflare
freq                            19
Name: company_permalink, dtype: object

In [10]:
set(companies['permalink'].unique()).difference(set(rounds2['company_permalink'].unique()))

set()

In [11]:
master_frame=pd.merge(rounds2,companies,how='left',left_on='company_permalink',right_on='permalink')
len(master_frame.index)

114949

In [12]:
master_frame.isnull().sum(axis=0)

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funding_round_code         83809
funded_at                      0
raised_amount_usd          19990
permalink                      0
name                           1
homepage_url                6134
category_list               3410
status                         0
country_code                8678
state_code                 10946
region                     10167
city                       10164
founded_at                 20521
dtype: int64

In [13]:
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)),2))

company_permalink           0.00
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
permalink                   0.00
name                        0.00
homepage_url                5.34
category_list               2.97
status                      0.00
country_code                7.55
state_code                  9.52
region                      8.84
city                        8.84
founded_at                 17.85
dtype: float64


In [14]:
master_frame = master_frame.drop(['funding_round_code','funding_round_permalink','funded_at','permalink','homepage_url','state_code','region',
                                 'city','founded_at','status'],axis=1)

In [15]:
print(round(100*(master_frame.isnull().sum()/len(master_frame.index)),2))

company_permalink      0.00
funding_round_type     0.00
raised_amount_usd     17.39
name                   0.00
category_list          2.97
country_code           7.55
dtype: float64


In [16]:
master_frame = master_frame[~(master_frame['raised_amount_usd'].isnull() | master_frame['country_code'].isnull() | 
                             master_frame['category_list'].isnull())]

In [17]:
print(100*(len(master_frame.index)/114949))

77.01589400516751


In [18]:
master_frame.funding_round_type.value_counts()

venture                  47809
seed                     21095
debt_financing            6506
angel                     4400
grant                     1939
private_equity            1820
undisclosed               1345
convertible_note          1320
equity_crowdfunding       1128
post_ipo_equity            598
product_crowdfunding       330
post_ipo_debt              151
non_equity_assistance       60
secondary_market            28
Name: funding_round_type, dtype: int64

In [19]:
master_frame=master_frame[(master_frame['funding_round_type']=='venture')
                         |(master_frame['funding_round_type']=='seed')
                         |(master_frame['funding_round_type']=='angel')
                         |(master_frame['funding_round_type']=='private_equity')]
master_frame.head()

,company_permalink,funding_round_type,raised_amount_usd,name,category_list,country_code
0,/organization/-fame,venture,10000000.0,#fame,Media,IND
2,/organization/-qounter,seed,700000.0,:Qounter,Application Platforms|Real Time|Social Network...,USA
4,/organization/0-6-com,venture,2000000.0,0-6.com,Curated Web,CHN
7,/organization/0ndine-biomedical-inc,seed,43360.0,Ondine Biomedical Inc.,Biotechnology,CAN
8,/organization/0ndine-biomedical-inc,venture,719491.0,Ondine Biomedical Inc.,Biotechnology,CAN


In [20]:
master_frame['raised_amount_usd']=master_frame['raised_amount_usd']/1000000
master_frame.head()

,company_permalink,funding_round_type,raised_amount_usd,name,category_list,country_code
0,/organization/-fame,venture,10.000000,#fame,Media,IND
2,/organization/-qounter,seed,0.700000,:Qounter,Application Platforms|Real Time|Social Network...,USA
4,/organization/0-6-com,venture,2.000000,0-6.com,Curated Web,CHN
7,/organization/0ndine-biomedical-inc,seed,0.043360,Ondine Biomedical Inc.,Biotechnology,CAN
8,/organization/0ndine-biomedical-inc,venture,0.719491,Ondine Biomedical Inc.,Biotechnology,CAN


In [21]:
round(master_frame.groupby('funding_round_type').raised_amount_usd.mean(),2)

funding_round_type
angel              0.97
private_equity    73.94
seed               0.75
venture           11.72
Name: raised_amount_usd, dtype: float64

In [22]:
master_frame=master_frame[master_frame['funding_round_type']=='venture']

In [23]:
master_frame = master_frame.drop(['funding_round_type'],axis=1)

In [24]:
top9 = master_frame.pivot_table(values = 'raised_amount_usd',index = 'country_code',aggfunc = 'sum')
top9 = top9.sort_values(by = 'raised_amount_usd',ascending = False)
top9 = top9.iloc[:9,]
top9

,raised_amount_usd
country_code,
USA,420068.029342
CHN,39338.918773
GBR,20072.813004
IND,14261.508718
CAN,9482.217668
FRA,7226.851352
ISR,6854.350477
DEU,6306.921981
JPN,3167.647127


In [25]:
master_frame=master_frame[(master_frame['country_code']=='USA')
                         |(master_frame['country_code']=='GBR')
                         |(master_frame['country_code']=='IND')]
master_frame.head()

,company_permalink,raised_amount_usd,name,category_list,country_code
0,/organization/-fame,10.0,#fame,Media,IND
10,/organization/0xdata,20.0,H2O.ai,Analytics,USA
11,/organization/0xdata,1.7,H2O.ai,Analytics,USA
12,/organization/0xdata,8.9,H2O.ai,Analytics,USA
22,/organization/1-mainstream,5.0,1 Mainstream,Apps|Cable|Distribution|Software,USA


In [26]:
# Extracting the primary vector values
master_frame['category_list'].apply(lambda x:x.split('|')[0])

0                 Media
10            Analytics
11            Analytics
12            Analytics
22                 Apps
              ...      
114920    Biotechnology
114922    Biotechnology
114923    Biotechnology
114934       E-Commerce
114936    Biotechnology
Name: category_list, Length: 38803, dtype: object

In [27]:
mapping = pd.read_csv('mapping.csv')

In [28]:
mapping.category_list.replace({'0':'na','2.na':'2.0'},regex = True)

0                   NaN
1                    3D
2           3D Printing
3         3D Technology
4            Accounting
             ...       
683           Wholesale
684    Wine And Spirits
685            Wireless
686               Women
687        Young Adults
Name: category_list, Length: 688, dtype: object

In [29]:
mapping.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [33]:
mapping = pd.melt(mapping, id_vars = ['category_list'], value_vars = ['Manufacturing','Automotive & Sports',
                                                                      'Cleantech / Semiconductors','Entertainment',
                                                                      'Health','Blanks','News, Search and Messaging',
                                                                      'Others','Social, Finance, Analytics, Advertising'])

In [35]:
mapping = mapping[~(mapping.values == 0)]
mapping = mapping.drop('value', axis = 1)
mapping = mapping.rename(columns = {"variable":"main_sector"})
mapping.head()

,category_list,main_sector
0,NaN,Manufacturing
0,NaN,Manufacturing
0,NaN,Manufacturing
0,NaN,Manufacturing
1,3D,Manufacturing


In [41]:
D1 = master_frame[(master_frame['country_code'] == 'USA') &
                 (master_frame['raised_amount_usd'] >= 5) &
                 (master_frame['raised_amount_usd'] <= 15)]
D1_gr = D1[['raised_amount_usd','main_sector']].groupby('main_sector').agg(['sum','count']).rename(
columns={'sum':'Total_amount','count':'Total_count'})
D1 = D1.merge(D1_gr, how = 'left', on = 'main_sector')
D1.head()

KeyError: "['main_sector'] not in index"